In [ ]:
import pandas as pd

In [ ]:
df_train = pd.read_csv('playground-series-s4e9/train.csv')
df_train

In [ ]:
df_train.head()

In [ ]:
df_train.isnull().sum()

In [ ]:
df_test = pd.read_csv('playground-series-s4e9/test.csv')
df_test.head()

In [ ]:
df_test.isnull().sum()

In [ ]:
import numpy as np

In [ ]:
df_train = df_train.drop(columns=['int_col'], axis=1)

In [ ]:
df_test = df_test.drop(columns=['int_col'], axis=1)

In [ ]:
df_train["fuel_type"].unique()

In [ ]:
from sklearn.impute import SimpleImputer

df_train['fuel_type'] = df_train['fuel_type'].replace('–', np.nan)
df_train['fuel_type'] = df_train['fuel_type'].replace('not supported', np.nan)
#ful_imper = SimpleImputer(strategy='most_frequent')
#df_train['fuel_type'] = ful_imper.fit_transform(df_train[['fuel_type']])

In [ ]:
df_train.isnull().sum()

In [ ]:
df_test.isnull().sum()

In [ ]:
def data_from_engine(df_test,df_train):
    df_test['horsepower'] = df_test['engine'].str.extract(r'(\d+.\d+)(?=HP)').astype(float)
    df_train['horsepower'] = df_train['engine'].str.extract(r'(\d+.\d+)(?=HP)').astype(float)
    df_train['engine_size'] = df_train['engine'].str.extract(r'(\d+.\d+)(?=L)').astype(float)
    df_test['engine_size'] = df_test['engine'].str.extract(r'(\d+.\d+)(?=L)').astype(float)
    df_test['cylinders'] = df_test['engine'].str.extract(r'(\d+)\s(Cylinder|V\d|Straight)')[0].astype(float)
    df_train['cylinders'] = df_train['engine'].str.extract(r'(\d+)\s(Cylinder|V\d|Straight)')[0].astype(float)
    df_train['horsepower'] = round(df_train['horsepower'].fillna(df_train['horsepower'].mean()))
    df_test['horsepower'] = round(df_test['horsepower'].fillna(df_test['horsepower'].mean()))
    df_train['engine_size'] = round(df_train['engine_size'].fillna(df_train['engine_size'].mean()))
    df_test['engine_size'] = round(df_test['engine_size'].fillna(df_test['engine_size'].mean()))
    df_test['cylinders'] = round(df_test['cylinders'].fillna(df_test['cylinders'].mean()))
    df_train['cylinders'] = round(df_train['cylinders'].fillna(df_train['cylinders'].mean()))
    df_train = df_train.drop('engine', axis = 1)
    df_test = df_test.drop('engine', axis = 1)
    return df_test,df_train

In [ ]:
def pre_precesing(df_test,df_train):
    moda = df_train['fuel_type'].mode()[0]
    df_train['fuel_type'] = df_train['fuel_type'].fillna(moda)
    df_test['fuel_type'] = df_train['fuel_type'].fillna(moda)
    df_train['accident'] = df_train['accident'].fillna('None reported')
    df_test['accident'] = df_test['accident'].fillna('None reported')
    df_train['clean_title'] = df_train['clean_title'].fillna('No')
    df_test['clean_title'] = df_test['clean_title'].fillna('No')
    df_test,df_train = data_from_engine(df_test,df_train)
    return df_test,df_train

In [ ]:
def clean_transmission(value):
    if pd.isna(value):
        return 'Other'

    val = value.lower()

    if 'cvt' in val:
        return 'CVT'
    elif 'manual' in val or 'm/t' in val or 'manual' in val:
        return 'Manual'
    elif 'auto-shift' in val or 'dct' in val or 'dual shift' in val:
        return 'Dual-Clutch'
    elif 'a/t' in val or 'automatic' in val:
        return 'Automatic'
    elif 'overdrive' in val:
        return 'Automatic'
    elif 'speed' in val and ('mt' in val or 'm/t' in val):
        return 'Manual'
    else:
        return 'Other'
df_train['transmission'] = df_train['transmission'].apply(clean_transmission)

In [ ]:
df_test, df_train = pre_precesing(df_test,df_train)

In [ ]:
df_train.isnull().sum()

In [ ]:
df_test.isnull().sum()

In [ ]:
df_test['fuel_type'].unique()

In [ ]:
df_test['accident'].unique()

In [ ]:
df_train

In [ ]:
df_train['brand'].unique()

In [ ]:
df_train['model'].unique()

In [ ]:
from sklearn.calibration import LabelEncoder


label_encoder = LabelEncoder()
df_train['model'] = label_encoder.fit_transform(df_train['model'])
df_train['model']

In [ ]:
df_train['brand'] = label_encoder.fit_transform(df_train['brand'])
df_train['brand']

In [ ]:
df_test['model'] = label_encoder.fit_transform(df_test['model'])
df_test['model']

In [ ]:
df_test['brand'] = label_encoder.fit_transform(df_test['brand'])
df_test['brand']

In [ ]:
df_train.dtypes

In [ ]:
X = df_train.drop('price', axis=1)
y = df_train['price']

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer

categorical_cols = X.select_dtypes(include='object').columns

encoder = ColumnTransformer(
    transformers=[
        ('onehot', OneHotEncoder(handle_unknown='ignore', drop='first'), categorical_cols)
    ],
    remainder='passthrough'  # lascia le altre colonne intatte
)

X_train_encoded = encoder.fit_transform(X_train)
X_test_encoded = encoder.transform(X_test)


In [ ]:
from sklearn.discriminant_analysis import StandardScaler
from sklearn.linear_model import Ridge
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import PolynomialFeatures


pipe = Pipeline([
    ('encode', encoder),
    ('scale', StandardScaler(with_mean = False)),
    ('poly', PolynomialFeatures(include_bias=False)),
    ('model', Ridge())  # Ridgeaggio per ridurre overfitting
])

In [ ]:
import scipy.stats as st
param_dist = {
    'poly__degree': st.randint(1, 4),   # grado tra 1 e 3
    'model__alpha': st.loguniform(1e-3, 1e2)  # regolarizzazione log‑uniforme
}

In [ ]:
from sklearn.model_selection import RandomizedSearchCV


search = RandomizedSearchCV(
    pipe,
    param_distributions=param_dist,
    n_iter=20,              
    scoring='neg_mean_squared_error',
    cv=5,
    random_state=42,
    n_jobs=-1
)

In [ ]:
search.fit(X_train, y_train)

print("Migliori parametri:", search.best_params_)
print("Best CV RMSE:", np.sqrt(-search.best_score_))